In [16]:
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
import numpy as np

import numpy as np
import rasterio

# define the year/years for the download
years = list(range(1986, 1987)) # from to
excluded_years = [2022] #no scenes
filtered_years = [year for year in years if year not in excluded_years]

for year in filtered_years: 

    # Define the band names
    band_names = ['blue_median','green_median', 'red_median', 'nir08_median', 'swir16_median', 'swir22_median']

    # Path to the input TIFF file
    input_tif_path = f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}.tif'

    # Path to the output TIFF file
    output_tif_path = f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}_clean.tif'

    # Open the TIFF file using rasterio
    with rasterio.open(input_tif_path) as src:
        # Read all bands of the raster data
        raster_data = src.read()

        # Replace specific values (e.g., 1e+20) with NaN for each band
        replaced_data = np.where(raster_data == 1e+20, np.nan, raster_data)

        # Create a new raster file with NaN values
        meta = src.meta.copy()  # Copy metadata from the source dataset
        meta.update(
            driver='GTiff',
            dtype=replaced_data.dtype,
            nodata=np.nan,  # Set NaN as the nodata value
            count=src.count,  # Number of bands
            tags={'band_names': ','.join(band_names)}  # Set band names as metadata
        )
        
        with rasterio.open(output_tif_path, 'w', **meta) as dst:
            dst.write(replaced_data)
            
    # Now, you can load the modified raster data back into a GeoDataFrame
    # and perform further geospatial operations if needed

